# Explore here

In [3]:
!pip install seaborn
!pip install scikit-learn
!pip install statsmodels
!pip install spacy
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [35]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.metrics import classification_report
import numpy as np
from sklearn.model_selection import GridSearchCV
import spacy
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from joblib import dump

In [5]:
url = "https://raw.githubusercontent.com/4GeeksAcademy/naive-bayes-project-tutorial/main/playstore_reviews.csv"
df = pd.read_csv(url)

In [6]:
df

,package_name,review,polarity
0,com.facebook.katana,privacy at least put some option appear offli...,0
1,com.facebook.katana,"messenger issues ever since the last update, ...",0
2,com.facebook.katana,profile any time my wife or anybody has more ...,0
3,com.facebook.katana,the new features suck for those of us who don...,0
4,com.facebook.katana,forced reload on uploading pic on replying co...,0
...,...,...,...
886,com.rovio.angrybirds,loved it i loooooooooooooovvved it because it...,1
887,com.rovio.angrybirds,all time legendary game the birthday party le...,1
888,com.rovio.angrybirds,ads are way to heavy listen to the bad review...,0
889,com.rovio.angrybirds,fun works perfectly well. ads aren't as annoy...,1


In [7]:
print(df.isnull().sum())

package_name    0
review          0
polarity        0
dtype: int64


In [8]:
df = df.drop(['package_name'], axis = 1)

In [9]:
df

,review,polarity
0,privacy at least put some option appear offli...,0
1,"messenger issues ever since the last update, ...",0
2,profile any time my wife or anybody has more ...,0
3,the new features suck for those of us who don...,0
4,forced reload on uploading pic on replying co...,0
...,...,...
886,loved it i loooooooooooooovvved it because it...,1
887,all time legendary game the birthday party le...,1
888,ads are way to heavy listen to the bad review...,0
889,fun works perfectly well. ads aren't as annoy...,1


In [10]:
df["review"] = df["review"].str.strip().str.lower()

In [11]:
# Dividimos los datos en conjuntos de entrenamiento y testeo
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['polarity'], test_size=0.2, random_state=42)

In [12]:
# Realizamos el conteo de palabras en los textos usando CountVectorizer
vectorizer = CountVectorizer(stop_words = "english")
X_train_vec = vectorizer.fit_transform(X_train).toarray()
X_test_vec = vectorizer.transform(X_test).toarray()

In [13]:
print(f'Bolsa de palabras que se usa para el entrenamiento:\n{vectorizer.get_feature_names_out()}')

Bolsa de palabras que se usa para el entrenamiento:
['000' '04' '0x' ... 'žŕ' 'žŕľ' 'ˇŕ']


In [14]:
X_test

709    love/hate has bug and security issues. i tried...
439    whatsapp i use this app now that blackberry me...
840                             usefully verry  nice app
720    fonts why in the heck is this thing analysing ...
39     app doesn't work after latest upgrade the face...
                             ...                        
433    app continuously losses connection, at times i...
773    way below expection. why does it lag so much? ...
25     can't install (error code: -505) have samsung ...
84     sort it out why can i not get my networks post...
10     what the heck?! can't get status updates to be...
Name: review, Length: 179, dtype: object

In [15]:
print(f'Email de prueba: {X_test.iloc[0]}')
print('\nPalabras del conjunto de entrenamiento que también aparecen en el email de test junto con su aparición:\n')
for i, cont in enumerate(X_test_vec[0]):
  if cont!=0:
    print(f'Palabra: "{vectorizer.get_feature_names_out()[i]}"')
    print(f'--> Aparece {cont} veces en el email.')

Email de prueba: love/hate has bug and security issues. i tried to report that facebook and google plus have security issues and it wouldn't allow me to do so! well i just did didn't i! ......

Palabras del conjunto de entrenamiento que también aparecen en el email de test junto con su aparición:

Palabra: "allow"
--> Aparece 1 veces en el email.
Palabra: "bug"
--> Aparece 1 veces en el email.
Palabra: "did"
--> Aparece 1 veces en el email.
Palabra: "didn"
--> Aparece 1 veces en el email.
Palabra: "facebook"
--> Aparece 1 veces en el email.
Palabra: "google"
--> Aparece 1 veces en el email.
Palabra: "hate"
--> Aparece 1 veces en el email.
Palabra: "issues"
--> Aparece 2 veces en el email.
Palabra: "just"
--> Aparece 1 veces en el email.
Palabra: "love"
--> Aparece 1 veces en el email.
Palabra: "plus"
--> Aparece 1 veces en el email.
Palabra: "report"
--> Aparece 1 veces en el email.
Palabra: "security"
--> Aparece 2 veces en el email.
Palabra: "tried"
--> Aparece 1 veces en el email.
P

In [16]:
# Inicializar y entrenar el clasificador Naive Bayes Multinomial
clf = MultinomialNB().fit(X_train_vec, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_MNB = clf.predict(X_test_vec)

# Evaluar el rendimiento del modelo
print('Para el modelo Multinominal')
print(classification_report(y_test, y_pred_MNB))

Para el modelo Multinominal
              precision    recall  f1-score   support

           0       0.84      0.90      0.87       126
           1       0.73      0.60      0.66        53

    accuracy                           0.82       179
   macro avg       0.79      0.75      0.77       179
weighted avg       0.81      0.82      0.81       179



In [17]:
# Inicializar y entrenar el clasificador Naive Bayes Gaussiano
clf_2 = GaussianNB().fit(X_train_vec, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_GNB = clf_2.predict(X_test_vec)

# Evaluar el rendimiento del modelo
print('Para el modelo Gaussiano')
print(classification_report(y_test, y_pred_GNB))

Para el modelo Gaussiano
              precision    recall  f1-score   support

           0       0.85      0.88      0.86       126
           1       0.69      0.62      0.65        53

    accuracy                           0.80       179
   macro avg       0.77      0.75      0.76       179
weighted avg       0.80      0.80      0.80       179



In [18]:
# Inicializar y entrenar el clasificador Naive Bayes de Bernoulli
clf_3 = BernoulliNB().fit(X_train_vec, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_BNB = clf_3.predict(X_test_vec)

# Evaluar el rendimiento del modelo
print('Para el modelo de Bernoulli')
print(classification_report(y_test, y_pred_BNB))

Para el modelo de Bernoulli
              precision    recall  f1-score   support

           0       0.79      0.93      0.85       126
           1       0.70      0.40      0.51        53

    accuracy                           0.77       179
   macro avg       0.74      0.66      0.68       179
weighted avg       0.76      0.77      0.75       179



## ANÁLISIS
#### Como era de esperar, el modelo multinominal es el que mejor resultados ha dado. Vamos a proceder a optimizarlo.

In [19]:
# Ajustar CountVectorizer con bigramas y trigramas, y stop words personalizados
vec_model = CountVectorizer(ngram_range=(1, 5), stop_words="english") # Con la incorporación de n-gramas vamos a tener en cuenta cadenas de palabras, no sólo palabras sueltas

X_train_vec = vec_model.fit_transform(X_train).toarray()
X_test_vec = vec_model.transform(X_test).toarray()

In [20]:
# Definir el clasificador y los parámetros para la búsqueda de cuadrícula
clf = MultinomialNB()
param_grid = {'alpha': [0.01, 0.1, 1, 10, 100]}

# Inicializar GridSearchCV
grid_search = GridSearchCV(clf, param_grid, cv=5)

# Entrenar utilizando la búsqueda de cuadrícula
grid_search.fit(X_train_vec, y_train)

# Ver el mejor valor de alpha y rendimiento
print("Mejor valor de alpha:", grid_search.best_params_)
print("Mejor puntaje:", grid_search.best_score_)

# Utilizar el mejor clasificador encontrado
clf_optimizado = grid_search.best_estimator_

Mejor valor de alpha: {'alpha': 10}
Mejor puntaje: 0.7640401851669456


In [21]:
# Inicializar y entrenar el clasificador Naive Bayes Multinomial
clf = MultinomialNB().fit(X_train_vec, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_MNB = clf.predict(X_test_vec)

# Evaluar el rendimiento del modelo
print('Para el modelo Multinominal optimizado')
print(classification_report(y_test, y_pred_MNB))

Para el modelo Multinominal optimizado
              precision    recall  f1-score   support

           0       0.84      0.96      0.90       126
           1       0.86      0.57      0.68        53

    accuracy                           0.84       179
   macro avg       0.85      0.76      0.79       179
weighted avg       0.85      0.84      0.83       179



## ANÁLISIS
#### El modelo ha mejorado sensiblemente
El incremento en la precisión de la clase 1 (comentarios positivos) a 0.86 y un recall de 0.57 indica que, aunque el modelo es muy preciso cuando predice comentarios positivos, todavía hay un número significativo de comentarios positivos que no está captando. El alto valor de precisión para la clase 0 (comentarios negativos) de 0.84 y un recall aún mayor de 0.96 muestra que el modelo es particularmente bueno para identificar los comentarios negativos correctamente.

La mejora en el puntaje F1 para ambas clases sugiere que hemos logrado un buen equilibrio entre la precisión y el recall, aunque aún hay margen de mejora, especialmente en el recall de la clase 1. El puntaje F1 es una medida armonizada de precisión y recall, y es particularmente útil para comparar modelos en tareas de clasificación con clases desbalanceadas. 

In [22]:
# Vamos a aplicar lematización para mejorar el rendimiento. Esto hará que palabra escritas en lenguaje informal puedan ser detectadas por el modelo
# Cargar el modelo de spacy
nlp = spacy.load('en_core_web_sm') # Lo hemos descargado desde la consola

# Función para lematizar el texto
def lematizar_texto(texto):
    doc = nlp(texto)
    lemas = [token.lemma_ for token in doc if not token.is_punct]
    return ' '.join(lemas)

# Aplicar la lematización a los conjuntos de entrenamiento y prueba
X_train_lematizado = [lematizar_texto(texto) for texto in X_train]
X_test_lematizado = [lematizar_texto(texto) for texto in X_test]

# Debemos vectorizar de nuevo los textos lematizados antes de entrenar el modelo
vec_model = CountVectorizer(ngram_range=(1, 5), stop_words="english")
X_train_vec = vec_model.fit_transform(X_train_lematizado).toarray()
X_test_vec = vec_model.transform(X_test_lematizado).toarray()

In [27]:
# PRobamos de nuevo tras lematizar
# Inicializar y entrenar el clasificador Naive Bayes Multinomial
clf_lema = MultinomialNB().fit(X_train_vec, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_MNB_lema = clf_lema.predict(X_test_vec)

# Evaluar el rendimiento del modelo
print('Para el modelo Multinominal optimizado tras lematizar')
print(classification_report(y_test, y_pred_MNB_lema))

Para el modelo Multinominal optimizado tras lematizar
              precision    recall  f1-score   support

           0       0.83      0.94      0.88       126
           1       0.81      0.55      0.65        53

    accuracy                           0.83       179
   macro avg       0.82      0.75      0.77       179
weighted avg       0.82      0.83      0.82       179



## ANÄLISIS
Observamos que el rendimiento ha bajado tras lematizar, por lo cual preferimos el modelo optimizado previo

In [23]:
# Convertir los conjuntos de datos a DMatrix, un formato interno de XGBoost que es más eficiente
dtrain = xgb.DMatrix(X_train_vec, label=y_train)
dtest = xgb.DMatrix(X_test_vec, label=y_test)

# Definir los parámetros para XGBoost
param = {'max_depth': 6, 'eta': 0.3, 'objective': 'binary:logistic'}
num_round = 100  # Número de rondas de entrenamiento

# Entrenar el modelo
bst = xgb.train(param, dtrain, num_round)

# Hacer predicciones
y_pred_xgb = bst.predict(dtest)
# Convertir las probabilidades a clases binarias
y_pred_xgb = [1 if prob > 0.5 else 0 for prob in y_pred_xgb]

# Evaluar el rendimiento
print(classification_report(y_test, y_pred_xgb))

              precision    recall  f1-score   support

           0       0.86      0.86      0.86       126
           1       0.67      0.68      0.67        53

    accuracy                           0.80       179
   macro avg       0.77      0.77      0.77       179
weighted avg       0.81      0.80      0.80       179



## ANÁLISIS
Con el XGBoost encontramos un rendimiento ligeramente inferior al uso del MultinominalNB. Inicialmente pensamos que podría ofrecer un rendimiento superior al RandomForest, debido a que funciona de manera similar pero asignando más peso a los errores para mejorar el rendimiento. 
Procedemos a probar con un RandomForest.

In [24]:
# Inicializar el clasificador de Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el clasificador de Random Forest
rf_clf.fit(X_train_vec, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_rf = rf_clf.predict(X_test_vec)

# Evaluar el rendimiento del modelo
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.86      0.83      0.84       126
           1       0.62      0.68      0.65        53

    accuracy                           0.78       179
   macro avg       0.74      0.75      0.75       179
weighted avg       0.79      0.78      0.78       179



## ANÁLISIS
EL Random Forest muestra un rendimiento menor a nuestro mejor MultinominalNB. No obstante, aún podemos mejorar el rendimiento del RF con una GridSearch o una RandomSearch

In [25]:
# Definir el modelo y los parámetros para la búsqueda
rf_clf = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Inicializar GridSearchCV
grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')

# Entrenar utilizando la búsqueda de cuadrícula
grid_search.fit(X_train_vec, y_train)

# Mejores parámetros y puntaje
print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntaje de cross-validation:", grid_search.best_score_)

# Entrenar y evaluar el modelo optimizado
best_rf_clf = grid_search.best_estimator_
y_pred_optimized = best_rf_clf.predict(X_test_vec)
print(classification_report(y_test, y_pred_optimized))

Mejores parámetros: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Mejor puntaje de cross-validation: 0.7794444991628091
              precision    recall  f1-score   support

           0       0.86      0.83      0.84       126
           1       0.62      0.68      0.65        53

    accuracy                           0.78       179
   macro avg       0.74      0.75      0.75       179
weighted avg       0.79      0.78      0.78       179



In [28]:
# Conteo de clases para optimizar el parámetro class_weight
clases, conteo_clases = np.unique(y_train, return_counts=True)
conteo_clases_dict = dict(zip(clases, conteo_clases))

print("Conteo de elementos por clase en y_train:", conteo_clases_dict)

Conteo de elementos por clase en y_train: {0: 458, 1: 254}


In [30]:
# Ajustar CountVectorizer con n-gramas y stop words
vec_model = CountVectorizer(ngram_range=(1, 5), stop_words="english") # Con la incorporación de n-gramas vamos a tener en cuenta cadenas de palabras, no sólo palabras sueltas

X_train_vec = vec_model.fit_transform(X_train).toarray()
X_test_vec = vec_model.transform(X_test).toarray()

In [32]:
# Probams de nuevo con RF pero sin lematizar
# Inicializar el clasificador de Random Forest
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el clasificador de Random Forest
rf_clf.fit(X_train_vec, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred_rf = rf_clf.predict(X_test_vec)

# Evaluar el rendimiento del modelo
print('Para el RF sin lematizar')
print(classification_report(y_test, y_pred_rf))

Para el RF sin lematizar
              precision    recall  f1-score   support

           0       0.84      0.82      0.83       126
           1       0.59      0.62      0.61        53

    accuracy                           0.76       179
   macro avg       0.71      0.72      0.72       179
weighted avg       0.76      0.76      0.76       179



## ANÁLISIS
El modelo Multinomial Naive Bayes antes de la lematización tenía una precisión del 84%, la cual es superior a la de RandomForest sin lematización.

La lematización, en el caso del modelo Naive Bayes, había llevado a una precisión del 83%, que todavía supera al RandomForest sin lematización.

El rendimiento de RandomForest sin lematización (76% de precisión) es ligeramente inferior al de RandomForest con lematización (78% de precisión).

Esto sugiere que, al menos para este conjunto de datos y tarea, el modelo Multinomial Naive Bayes supera al RandomForest, independientemente de la aplicación de la lematización.

In [34]:
# Vamos a probar con un RF ajustando los pesos
# Calcular pesos inversos de las clases
weights = {0: 1, 1: 458 / 254}

rf_clf_weighted = RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_leaf=1, min_samples_split=2, class_weight=weights, random_state=42)
rf_clf_weighted.fit(X_train_vec, y_train)
y_pred_rf_weighted = rf_clf_weighted.predict(X_test_vec)

print('Para el RF con los pesos ajustados')
print(classification_report(y_test, y_pred_rf_weighted))

Para el RF con los pesos ajustados
              precision    recall  f1-score   support

           0       0.86      0.74      0.79       126
           1       0.54      0.72      0.61        53

    accuracy                           0.73       179
   macro avg       0.70      0.73      0.70       179
weighted avg       0.76      0.73      0.74       179



## ANÁLISIS
La precisión para la clase 0 (comentarios negativos) se mantuvo alta en 0.86, pero su recall disminuyó a 0.74, lo que indica que el modelo se volvió un poco menos efectivo en identificar correctamente todos los comentarios negativos.

La precisión para la clase 1 aumentó a 0.54 y su recall a 0.72. Esto significa que, al ajustar los pesos, el modelo ha mejorado significativamente su capacidad para identificar comentarios positivos, aunque a costa de etiquetar incorrectamente más comentarios negativos como positivos (lo que explica la disminución de la precisión en esta clase).

La precisión general (accuracy) del modelo disminuyó ligeramente a 0.73, reflejando el compromiso entre mejorar el recall de la clase minoritaria y mantener la alta precisión para la clase mayoritaria.

## Conclusiones y Reflexiones
Mejora en la Detección de la Clase Minoritaria: Ajustar el class_weight ha demostrado ser efectivo para mejorar la detección de comentarios positivos, que era el objetivo al considerar el desbalance de clases.

Selección del Modelo: Aunque el ajuste de pesos ayudó a mejorar el recall para los comentarios positivos, el modelo Multinomial Naive Bayes sin lematización sigue mostrando un mejor balance general entre precisión y recall, con una precisión (accuracy) global superior.

In [36]:
dump(clf_optimizado, 'mejor_modelo_Multinb_42.joblib')

['mejor_modelo_Multinb_42.joblib']

Como el modelo con mejor rendimiento es el MultinomialNB sin lematizar, es el que hemos guardado.